# -----------------PROYECTO FINAL-----------------

## 1. IMPORTAR LIBRERIAS

In [1]:
from keras.src.saving.saving_api import load_model
import cv2
import mediapipe as mp
import numpy as np
import os
import tensorflow as tf
import sys
from collections import deque  

### Este Programa no cambia del me_8 hasta la etapa de entrenamiento en adelante

## 2. INICIALIZAR MEDIAPIPE

In [2]:
# Configuración inicial global
# Importa el módulo de manos de MediaPipe para la detección de landmarks
mp_hands = mp.solutions.hands

# Inicializa el detector de manos con configuración específica:
hands = mp_hands.Hands(
    static_image_mode=False,  # Modo video (mejor para secuencias)
    max_num_hands=2,          # Máximo número de manos a detectar
    min_detection_confidence=0.45,  # Confianza mínima para detección inicial
    min_tracking_confidence=0.45,   # Confianza mínima para seguimiento continuo
    model_complexity=1        # Balance entre precisión y rendimiento (0=ligero, 1=medio, 2=completo)
)

# Utilidades para dibujar los landmarks y conexiones en la imagen
mp_draw = mp.solutions.drawing_utils
# Directorio donde se almacenarán los datos del dataset
dataset_dir = "dataset_11"
# Ruta donde se guardará/recuperará el modelo entrenado
model_path = "gesture_model_me_10.h5"
# Longitud de las secuencias temporales (número de frames por muestra)
sequence_length = 30
# Número total de landmarks (21 puntos por mano * 3 coordenadas (x,y,z) * 2 manos)
total_landmarks = 126
# Lista para almacenar los nombres de los gestos/clases
gestures = []
# Variables para normalización de datos (media y desviación estándar)
X_mean = None
X_std = None


num_camara = 0

## 3. FUNCIONES PRINCIPALES

In [3]:
# Funciones principales
def init_system():
    global gestures # Accede a la variable global para almacenar los gestos

    # Crea el directorio principal del dataset si no existe
    # exist_ok=True evita errores si el directorio ya existe    
    os.makedirs(dataset_dir, exist_ok=True)

    # Obtiene la lista de gestos existentes del disco
    gestures = get_existing_gestures()
    
# Función para obtener los gestos ya registrados
def get_existing_gestures():
    #Lista comprensiva que recorre todos los elementos en el directorio del dataset
    return [d for d in os.listdir(dataset_dir) 
            # Verifica si es un directorio (y no un archivo)
           if os.path.isdir(os.path.join(dataset_dir, d))]

## 4. DETECCION DE MANO

In [4]:
def detect_hands():
    # Muestra mensaje de inicio para el usuario
    print("\nIniciando detección de manos. Presiona 'ESC' para salir.")
    # Inicializa la captura de video desde la cámara predeterminada (índice 0)
    cap = cv2.VideoCapture(0)

    # Bucle principal de procesamiento de frames
    while True:
        # Lee un frame de la cámara
        ret, frame = cap.read()
        # Verifica si la lectura del frame fue exitosa
        if not ret:
            break  # Sale del bucle si hay error

        # Convierte el frame de BGR (OpenCV) a RGB (MediaPipe)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Procesa el frame con el modelo de detección de manos
        results = hands.process(rgb_frame)

        # Si se detectaron landmarks de manos
        if results.multi_hand_landmarks:
            # Itera sobre cada mano detectada
            for hand_landmarks in results.multi_hand_landmarks:
                # Dibuja los landmarks y conexiones en el frame
                mp_draw.draw_landmarks(
                    frame, 
                    hand_landmarks, 
                    mp_hands.HAND_CONNECTIONS  # Dibuja conexiones entre landmarks
                )

        # Muestra el frame procesado en una ventana
        cv2.imshow("Detección de Manos", frame)
        
        # Verifica si se presionó la tecla ESC (código 27)
        if cv2.waitKey(1) & 0xFF == 27:
            break  # Sale del bucle

    # Libera los recursos de la cámara
    cap.release()
    
    # Cierra todas las ventanas de OpenCV
    cv2.destroyAllWindows()

## 5. RECOLLECION DE DATOS 

In [5]:
def collect_data():
    global gestures  # Accede a la variable global de gestos
    
    # Solicita datos al usuario
    gesture = input("\nIngrese la palabra o letra para la cual desea recolectar datos: ").upper()
    num_sequences = int(input("Ingrese el número de secuencias a capturar (recomendado: 50): "))
    
    # Crea directorio para almacenar los datos
    save_dir = os.path.join(dataset_dir, gesture)
    os.makedirs(save_dir, exist_ok=True)  # Crea el directorio si no existe

    # Mensajes informativos
    print(f"\nRecolectando datos para el gesto '{gesture}'. Presiona 'ESC' para cancelar.")
    print("Mantenga la seña frente a la cámara...")
    
    # Inicializa captura de video
    cap = cv2.VideoCapture(0)
    sequence = []  # Almacena la secuencia actual
    counter = 0    # Contador de secuencias guardadas

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Procesamiento del frame
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)

        if results.multi_hand_landmarks:
            # Extrae landmarks de hasta 2 manos
            all_landmarks = []
            for hand in results.multi_hand_landmarks[:2]:  # Máximo 2 manos
                for lm in hand.landmark:
                    all_landmarks.extend([lm.x, lm.y, lm.z])  # Coordenadas normalizadas
            
            # Rellena con ceros si solo hay una mano detectada
            if len(results.multi_hand_landmarks) < 2:
                all_landmarks += [0.0] * 63  # 21 landmarks * 3 coordenadas
            
            sequence.append(all_landmarks)  # Agrega a la secuencia
            
            # Dibuja landmarks en el frame
            for hand_landmarks in results.multi_hand_landmarks:
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # Guarda la secuencia cuando alcanza la longitud deseada
        if len(sequence) == sequence_length:
            np.save(os.path.join(save_dir, f"secuencia_{counter}.npy"), sequence)
            counter += 1
            sequence = []  # Reinicia la secuencia
            print(f"Secuencias capturadas: {counter}/{num_sequences}")

        # Muestra vista previa
        cv2.imshow("Recolección de Datos", frame)
        
        # Condiciones de salida: ESC o completar secuencias
        if cv2.waitKey(1) & 0xFF == 27 or counter >= num_sequences:
            break

    # Libera recursos
    cap.release()
    cv2.destroyAllWindows()
    
    # Actualiza lista de gestos
    gestures = get_existing_gestures()
    print(f"\nSe recolectaron {counter} secuencias para el gesto '{gesture}'")

## 6. CARGA DE DATOS

In [6]:
def load_data():
    # Listas para almacenar características (X) y etiquetas (y)
    X = []
    y = []
    
    # Iterar sobre cada gesto con su índice como etiqueta
    for label_idx, gesture in enumerate(gestures):
        # Construir ruta al directorio del gesto
        gesture_dir = os.path.join(dataset_dir, gesture)
        
        # Listar solo archivos .npy válidos (30,126)
        sequences = [f for f in os.listdir(gesture_dir) if f.endswith('.npy')]
        
        # Procesar cada secuencia del gesto actual
        for seq_file in sequences:
            seq_path = os.path.join(gesture_dir, seq_file)
            sequence = np.load(seq_path)  # Cargar secuencia desde disco
            
            # Validar dimensiones del dato (30 frames × 126 landmarks)
            if sequence.shape == (sequence_length, total_landmarks):
                X.append(sequence)       # Agregar secuencia a características
                y.append(label_idx)      # Agregar índice como etiqueta
    
    # Convertir a arrays numpy y retornar con nombres de gestos
    return np.array(X), np.array(y), gestures

## 7. ENTRENAMIENTO DEL MODELO

In [7]:
def train_model():
    global X_mean, X_std, gestures  # Accede a variables globales para normalización y gestos
    
    # 1. Verificación inicial de datos
    gestures = get_existing_gestures()  # Obtiene lista de gestos desde el directorio
    if not gestures:  # Valida existencia de datos
        print("\nNo hay datos recolectados. Primero recolecte datos de gestos.")
        return

    # 2. Carga y preparación de datos
    print("\nCargando datos y preparando el entrenamiento...")
    X, y, gestures = load_data()  # Carga secuencias y etiquetas desde .npy
    y = tf.keras.utils.to_categorical(y)  # Convierte etiquetas a one-hot encoding

    # 3. Normalización de datos
    X_mean = np.mean(X, axis=(0, 1))  # Calcula media por landmark
    X_std = np.std(X, axis=(0, 1))  # Calcula desviación estándar
    X = (X - X_mean) / X_std  # Estandarización de características

    # 4. Persistencia de parámetros
    np.savez('normalization_params.npz', mean=X_mean, std=X_std)  # Guarda stats para inferencia
    
    # 5. Construcción del modelo
    inputs = tf.keras.Input(shape=(sequence_length, total_landmarks))  # Capa de entrada
    x = tf.keras.layers.Conv1D(64, 3, activation='relu', padding='same')(inputs)  # Extracción de patrones locales
    x = tf.keras.layers.MaxPooling1D(2)(x)  # Reducción dimensional
    x = tf.keras.layers.LSTM(64, return_sequences=False)(x)  # Procesamiento temporal
    outputs = tf.keras.layers.Dense(len(gestures), activation='softmax')(x)  # Clasificación final
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)  # Ensamblado del modelo

    # 6. Configuración del entrenamiento
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),  # Optimizador adaptativo
        loss='categorical_crossentropy',  # Función de pérdida para clasificación
        metrics=['accuracy']  # Métrica principal
    )

    # 7. Proceso de entrenamiento
    print("\nIniciando entrenamiento...")
    history = model.fit(
        X, y,
        epochs=50,  # Máximo de iteraciones
        batch_size=32,  # Tamaño de lote
        validation_split=0.2,  # 20% datos para validación
        callbacks=[tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)],  # Control overfitting
        verbose=1  # Mostrar progreso
    )

    # 8. Persistencia del modelo
    model.save(model_path)  # Guardado en formato HDF5
    print(f"\nModelo guardado en {model_path}")
    
    # 9. Conversión para despliegue
    converter = tf.lite.TFLiteConverter.from_keras_model(model)  # Inicializa conversor
    converter.target_spec.supported_ops = [  # Configura compatibilidad
        tf.lite.OpsSet.TFLITE_BUILTINS,
        tf.lite.OpsSet.SELECT_TF_OPS
    ]
    converter._experimental_lower_tensor_list_ops = False  # Para compatibilidad con LSTM
    
    try:  # Manejo de errores en conversión
        tflite_model = converter.convert()
        with open('model_quantized.tflite', 'wb') as f:
            f.write(tflite_model)
        print("\nModelo TFLite exportado exitosamente")
    except Exception as e:
        print(f"\nError en conversión TFLite: {str(e)}")
    
    # 10. Reporte final
    val_accuracy = history.history['val_accuracy'][-1]  # Última métrica de validación
    print(f"Precisión de validación final: {val_accuracy:.2%}")

## TF LITE

In [8]:
def convert_to_tflite():
    try:
        # PASO 1: Cargar el modelo entrenado (ej: 'mi_modelo.h5')
        model = tf.keras.models.load_model(model_path)
        # Imagina que model_path es como la dirección de tu casa donde guardaste el modelo
        
        # PASO 2: Preparar el "traductor" (conversor)
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        # Crear un conversor especializado para tu modelo
        
        # PASO 3: Configurar compatibilidad para LSTM (¡CRUCIAL!)
        converter.target_spec.supported_ops = [
            tf.lite.OpsSet.TFLITE_BUILTINS,  # Operaciones básicas de TFLite
            tf.lite.OpsSet.SELECT_TF_OPS     # Operaciones especiales de TensorFlow
        ]
        # Esto es como decirle al traductor: "Usa el diccionario básico y agrega palabras técnicas"
        
        converter._experimental_lower_tensor_list_ops = False
        # Desactiva modificaciones automáticas en las operaciones de listas (necesario para LSTM)
        
        converter.allow_custom_ops = True
        # Permite operaciones personalizadas (como un traductor que acepta jerga local)
        
        # PASO 4: Realizar la conversión
        tflite_model = converter.convert()
        # ¡Traducción completada! Ahora tenemos el modelo en "idioma móvil"
        
        # PASO 5: Guardar el nuevo modelo
        with open('model_quantized.tflite', 'wb') as f:
            f.write(tflite_model)
        # Guardamos el libro traducido en un nuevo archivo .tflite
            
        print("\n✅ Conversión a TFLite exitosa!")
        # ¡Éxito! El modelo ahora es compatible con apps móviles
        
    except Exception as e:
        print(f"\n❌ Error en conversión: {str(e)}")
        print("Posibles soluciones:")
        print("1. Verifique que el modelo .h5 existe")
        print("2. Actualice TensorFlow: pip install --upgrade tensorflow")
        print("3. Reinicie el runtime/kernel")

def representative_dataset_gen():
    # Genera 100 muestras de datos sintéticos
    for _ in range(100):
        # Crea un tensor de ejemplo con forma (1, 30, 225) si sequence_length=30 y total_landmarks=225
        ejemplo = np.random.randn(1, sequence_length, total_landmarks).astype(np.float32)
        yield [ejemplo]
        # Entrega los datos de ejemplo al conversor como un lote de entrada

## 8. EVALUACION DEL MODELO

In [9]:
# ------------------ IMPORTS ADICIONALES NECESARIOS ------------------
from collections import deque  # Cola eficiente tipo FIFO (First-In First-Out)
from threading import Thread   # Para ejecutar tareas en paralelo
from queue import Queue        # Colas seguras para hilos (thread-safe)

# Ejemplo: Imagina estos imports como contratar especialistas:
# - deque: Un organizador de tareas
# - Thread: Un equipo de trabajadores
# - Queue: Cintas transportadoras entre estaciones de trabajo

# ------------------ SECCIÓN DE CONFIGURACIÓN ------------------
# Creación de 3 cintas transportadoras (colas) con capacidades diferentes
frame_queue = Queue(maxsize=30)      # Almacena hasta 30 fotogramas
landmark_queue = Queue(maxsize=20)   # Almacena hasta 20 conjuntos de landmarks
prediction_queue = Queue(maxsize=10) # Almacena hasta 10 predicciones

# Analogía: Una fábrica con 3 cintas transportadoras:
# 1. Para materias primas (frames)
# 2. Para piezas procesadas (landmarks)
# 3. Para productos terminados (predicciones)

# ------------------ FUNCIÓN CAMERA_CAPTURE ------------------
def camera_capture(cap):
    while True:  # Bucle infinito
        ret, frame = cap.read()  # Leer cámara (ej: 640x480)
        if ret:  # Si el frame es válido
            # Reducir resolución para ahorrar procesamiento (320x240)
            resized_frame = cv2.resize(frame, (320, 240))
            frame_queue.put(resized_frame)  # Poner en la cinta transportadora

# Ejemplo práctico:
# - Frame original: 640x480 (307,200 píxeles)
# - Frame redimensionado: 320x240 (76,800 píxeles) -> 75% menos datos

# ------------------ FUNCIÓN LANDMARK_PROCESSING ------------------
def landmark_processing():
    while True:
        frame = frame_queue.get()  # Tomar frame de la cinta
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convertir color
        results = hands.process(rgb_frame)  # Detectar manos con MediaPipe
        
        # Crear contenedor para 21 landmarks * 2 manos * 3 coordenadas (x,y,z)
        all_landmarks = np.zeros(total_landmarks, dtype=np.float32)
        
        if results.multi_hand_landmarks:  # Si detecta manos
            idx = 0  # Puntero de posición en el array
            # Procesar máximo 2 manos ([:2]) por si hay dos en pantalla
            for hand in results.multi_hand_landmarks[:2]:
                for lm in hand.landmark:  # Por cada landmark de la mano
                    # Almacenar coordenadas normalizadas
                    all_landmarks[idx:idx+3] = [lm.x, lm.y, lm.z]
                    idx += 3  # Mover puntero 3 posiciones
        
        landmark_queue.put(all_landmarks)  # Pasar a la siguiente cinta

# Ejemplo de datos:
# all_landmarks = [0.5, 0.3, 0.1, 0.6, 0.2, 0.9, ...]  # 126 valores en total

# ------------------ FUNCIÓN MODEL_INFERENCE ------------------
def model_inference(interpreter, input_details, output_details):
    sequence = deque(maxlen=sequence_length)  # Historial de últimos N landmarks
    
    while True:
        data = landmark_queue.get()  # Obtener landmarks de la cinta
        if data is not None:
            sequence.append(data)  # Agregar al historial
            sequence = sequence[-sequence_length:]  # Mantener sólo los últimos
            
            if len(sequence) == sequence_length:  # Cuando tenemos una secuencia completa
                # Normalización de datos (restar media y dividir por desviación)
                seq_array = (np.array(sequence) - X_mean) / X_std
                # Reformatear para el modelo: (1, 30, 126) si sequence_length=30
                input_data = seq_array.reshape(1, sequence_length, total_landmarks).astype(np.float32)
                
                # Ejecutar modelo TFLite
                interpreter.set_tensor(input_details[0]['index'], input_data)
                interpreter.invoke()  # Lanzar cálculo
                prediction = interpreter.get_tensor(output_details[0]['index'])[0]
                
                # Obtener resultados
                predicted_class = np.argmax(prediction)  # Índice de mayor probabilidad
                confidence = np.max(prediction)  # Valor de confianza
                prediction_queue.put((gestures[predicted_class], confidence))

# Ejemplo de predicción:
# prediction = [0.1, 0.8, 0.05, 0.05] -> predicted_class=1 (gestures[1]="Hola")
# confidence=0.8 (80% de seguridad)


In [10]:
from threading import Thread

def evaluate():
    # 1. Verificación inicial del modelo
    if not os.path.exists("model_quantized.tflite"):
        print("\n¡Primero debe entrenar y convertir el modelo!")
        return
    # Ejemplo: Es como querer usar un teléfono sin batería, primero debes cargarlo

    # 2. Carga de parámetros y modelo
    try:
        # 2.1 Cargar estadísticas de normalización (media y desviación)
        with np.load('normalization_params.npz') as data:
            X_mean = data['mean']  # Ej: [0.45, 0.32, ...] (126 valores)
            X_std = data['std']    # Ej: [0.12, 0.08, ...]
        
        # 2.2 Configurar intérprete de TensorFlow Lite
        interpreter = tf.lite.Interpreter(model_path="model_quantized.tflite")
        interpreter.allocate_tensors()  # Preparar memoria para el modelo
        # Analogía: Cargar un videojuego y asignar memoria de la consola
        
        input_details = interpreter.get_input_details()[0]  # Detalles entrada
        output_details = interpreter.get_output_details()[0]  # Detalles salida
    except Exception as e:
        print(f"\nError crítico: {str(e)}")  # Ej: Si el archivo .npz no existe
        return

    # 3. Configuración de la cámara
    cap = cv2.VideoCapture(0)  # Iniciar cámara (ID 0 = cámara predeterminada)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)   # Ancho de 640px
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)  # Alto de 480px
    # Ejemplo: Configurar una cámara de seguridad con resolución específica

    # 4. Variables de estado
    current_gesture = None  # Almacena la última seña detectada
    current_confidence = 0.0  # Nivel de confianza (0.0 a 1.0)
    sequence = deque(maxlen=sequence_length)  # Buffer de últimos N frames
    frame_counter = 0  # Contador para saltar frames

    # 5. Bucle principal de procesamiento
    while True:
        # 5.1 Capturar frame
        ret, frame = cap.read()  # Leer frame de la cámara
        if not ret:  # Si hay error en la captura
            print("\nError en captura de frame")
            break
            
        frame_counter += 1  # Incrementar contador de frames
        
        # 5.2 Procesar cada 2 frames (reducción de carga de procesamiento)
        if frame_counter % 2 != 0:
            continue  # Saltar frames impares
        # Ejemplo: Analizar 15 fps en lugar de 30 fps

        # 5.3 Detección de landmarks
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convertir a RGB
        results = hands.process(rgb_frame)  # Procesar con MediaPipe
        
        if results.multi_hand_landmarks:  # Si detecta manos
            # 5.4 Extraer coordenadas de landmarks
            landmarks = []
            for hand in results.multi_hand_landmarks[:2]:  # Máximo 2 manos
                for lm in hand.landmark:
                    landmarks.extend([lm.x, lm.y, lm.z])  # Aplanar coordenadas
            # Ejemplo de datos: [0.5, 0.3, 0.1, 0.7, 0.2, ...] (126 valores)
            
            # 5.5 Rellenar con ceros si solo hay una mano detectada
            if len(landmarks) < total_landmarks:
                landmarks += [0.0] * (total_landmarks - len(landmarks))
            
            sequence.append(landmarks)  # Agregar al buffer circular

            # 5.6 Dibujar landmarks en el frame
            for hand_landmarks in results.multi_hand_landmarks:
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # 5.7 Realizar predicción cada 30 frames (~1 segundo)
            if len(sequence) == sequence_length and frame_counter % 30 == 0:
                try:
                    # Preprocesamiento de datos
                    seq_array = np.array(sequence)  # Convertir a numpy array
                    seq_array = (seq_array - X_mean) / (X_std + 1e-7)  # Normalización
                    
                    # Reformatear para el modelo (1, 30, 126)
                    input_data = seq_array.reshape(1, sequence_length, total_landmarks).astype(np.float32)
                    
                    # 5.8 Ejecutar inferencia del modelo
                    interpreter.set_tensor(input_details['index'], input_data)
                    interpreter.invoke()  # Lanzar cálculo
                    prediction = interpreter.get_tensor(output_details['index'])[0]
                    
                    # 5.9 Procesar resultados
                    predicted_idx = np.argmax(prediction)  # Índice de mayor probabilidad
                    confidence = np.max(prediction)  # Valor de confianza
                    
                    # 5.10 Actualizar UI solo si confianza > 75%
                    if confidence > 0.75:
                        current_gesture = gestures[predicted_idx]
                        current_confidence = confidence
                        print(f"Detección: {current_gesture} ({confidence:.2%})")
                        
                        # Dibujar texto en el frame
                        cv2.putText(frame, f"{current_gesture} ({confidence:.2%})", 
                                   (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)

                except Exception as e:
                    print(f"Error en predicción: {str(e)}")
                    current_gesture = None

        # 5.11 Mostrar frame con resultados
        cv2.imshow("Predicciones", cv2.resize(frame, (640, 480)))
        
        # 5.12 Salir con tecla ESC
        if cv2.waitKey(1) & 0xFF == 27:
            break

    # 6. Liberar recursos
    cap.release()  # Apagar cámara
    cv2.destroyAllWindows()  # Cerrar ventanas
    print("\nSistema de evaluación detenido")

## 9. REENTRENAR GESTO

In [11]:
def retrain_gesture():
    global gestures
    if not gestures:
        print("\nNo hay gestos para reentrenar. Primero recolecte datos.")
        return

    print("\nGestos disponibles para reentrenar:")
    for i, gesture in enumerate(gestures):
        print(f"{i+1}. {gesture}")

    try:
        choice = int(input("\nSeleccione el número del gesto a reentrenar: ")) - 1
        if 0 <= choice < len(gestures):
            gesture = gestures[choice]
            gesture_dir = os.path.join(dataset_dir, gesture)
            
            for file in os.listdir(gesture_dir):
                os.remove(os.path.join(gesture_dir, file))
            
            print(f"\nDatos anteriores de '{gesture}' eliminados.")
            collect_data()
            train_model()
        else:
            print("\nSelección inválida.")
    except ValueError:
        print("\nPor favor, ingrese un número válido.")

## 10. MENU

In [12]:
# Menú principal
def main():
    init_system()
    
    while True:
        print("\n=== Sistema de Reconocimiento de Lenguaje de Señas ===")
        print("1. Detectar Manos")
        print("2. Recolectar Datos")
        print("3. Entrenar Modelo, y despues ir a convertir a TFlite")
        print("4. Evaluar")
        print("5. Reentrenar Gesto")
        print("6. Convertir a TFLite")  # Nueva opción
        print("7. Salir")
        
        choice = input("\nSeleccione una opción: ")
        
        if choice == '1':
            detect_hands()
        elif choice == '2':
            collect_data()
        elif choice == '3':
            train_model()
        elif choice == '4':
            evaluate()
        elif choice == '5':
            retrain_gesture()
        elif choice == '6':  # Nueva opción de conversión
            convert_to_tflite()
        elif choice == '7':
            print("\n¡Hasta luego!")
            break
        else:
            print("\nOpción inválida. Por favor, intente de nuevo.")

# MENU

In [13]:
if __name__ == "__main__":
    main()


=== Sistema de Reconocimiento de Lenguaje de Señas ===
1. Detectar Manos
2. Recolectar Datos
3. Entrenar Modelo, y despues ir a convertir a TFlite
4. Evaluar
5. Reentrenar Gesto
6. Convertir a TFLite
7. Salir

Cargando datos y preparando el entrenamiento...

Iniciando entrenamiento...
Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 285ms/step - accuracy: 0.0164 - loss: 1.5384 - val_accuracy: 0.0000e+00 - val_loss: 1.4505
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.0750 - loss: 1.4369 - val_accuracy: 0.0000e+00 - val_loss: 1.4151
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.2250 - loss: 1.3468 - val_accuracy: 0.0000e+00 - val_loss: 1.3807
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.4688 - loss: 1.2624 - val_accuracy: 0.1500 - val_loss: 1.3458
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - accuracy: 0.7711 - loss: 1.1734 - val_accuracy: 0.4500 - val_loss: 1.3100
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.8422 - loss:


Modelo guardado en gesture_model_me_10.h5
INFO:tensorflow:Assets written to: C:\Users\juanp\AppData\Local\Temp\tmp70f0nf6z\assets


INFO:tensorflow:Assets written to: C:\Users\juanp\AppData\Local\Temp\tmp70f0nf6z\assets


Saved artifact at 'C:\Users\juanp\AppData\Local\Temp\tmp70f0nf6z'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 30, 126), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 4), dtype=tf.float32, name=None)
Captures:
  1788561062048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1788561198240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1788561207392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1788561204928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1788561207040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1788561207920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1788561204576: TensorSpec(shape=(), dtype=tf.resource, name=None)

Modelo TFLite exportado exitosamente
Precisión de validación final: 90.00%

=== Sistema de Reconocimiento de Lenguaje de Señas ===
1. Detectar Manos
2. Recolectar Datos
3. Entrenar Modelo, y despues ir a convertir 

INFO:tensorflow:Assets written to: C:\Users\juanp\AppData\Local\Temp\tmpwqb7gwii\assets


...

INFO:tensorflow:Assets written to: C:\Users\juanp\AppData\Local\Temp\tmpw3p3awpp\assets


Saved artifact at 'C:\Users\juanp\AppData\Local\Temp\tmpw3p3awpp'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 30, 126), dtype=tf.float32, name='keras_tensor_13')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  1788731421472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1788731420768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1788731422704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1788731487008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1788731489120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1788731410912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1788731476272: TensorSpec(shape=(), dtype=tf.resource, name=None)

Modelo TFLite exportado exitosamente
Precisión de validación final: 100.00%

=== Sistema de Reconocimiento de Lenguaje de Señas ===
1. Detectar Manos
2. Recolectar Datos
3. Entrenar Modelo, y despues ir a conver

INFO:tensorflow:Assets written to: C:\Users\juanp\AppData\Local\Temp\tmpd6vgeq5_\assets


INFO:tensorflow:Assets written to: C:\Users\juanp\AppData\Local\Temp\tmpd6vgeq5_\assets


Saved artifact at 'C:\Users\juanp\AppData\Local\Temp\tmpd6vgeq5_'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 30, 126), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  1788786754832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1788786759760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1788786769264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1788786766976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1788786762928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1788786765744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1788786761872: TensorSpec(shape=(), dtype=tf.resource, name=None)

✅ Conversión a TFLite exitosa!

=== Sistema de Reconocimiento de Lenguaje de Señas ===
1. Detectar Manos
2. Recolectar Datos
3. Entrenar Modelo, y despues ir a convertir a TFlite
4. Evaluar
5. Reentrenar Gesto
6. 